In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import compress

from datetime import datetime
from dateutil.parser import parse

import math
import os
import copy
import pickle

import drugLookup

In [27]:
# Read in data (from pickle file)
file = open('train_set_filtered','rb')
train_set = pickle.load(file)

file = open('test_set_filtered','rb')
test_set = pickle.load(file)

In [97]:
d_train = list(train_set['drug_prediction'])
v_train = list(train_set['vendor_name'])

d_test = list(test_set['drug_prediction'])
v_test = list(test_set['vendor_name'])

## Compute 'inverse' emission probability
I.e. compute $P(v|d)$, the probability of each vendor given a specific drug.
Use $P(v~|~d) = \frac{P(d~|~v){P(v)}}{P(d)}$.

In [29]:
# Function gets tally of drugs sold for given vendor
def getVendorsForDrug(drug):
#     Args: Name of a drug (string)
#     Returns: series, representing proportion of total for each vendor
    drug_subset = train_set[train_set['drug_prediction'] == drug]
    drug_vendor_tally = drug_subset['vendor_name'].groupby(drug_subset['vendor_name']).count()
    return(drug_vendor_tally/np.sum(drug_vendor_tally))

# Function gets tally of drugs sold for given vendor
def getDrugsForVendor(vendor_name):
#     Args: Name of a vendor (string)
#     Returns: series, representing proportion of total for each drug
    vendor_subset = train_set[train_set['vendor_name'] == vendor_name]
    vendor_drug_tally = vendor_subset['drug_prediction'].groupby(vendor_subset['drug_prediction']).count()
    return(vendor_drug_tally/np.sum(vendor_drug_tally))

In [30]:
# Get list of vendors in the data frame
_ , vendor_list = pd.factorize(v_train, sort = True)

# Factorize drug_predictions
_ , drug_list = pd.factorize(d_train, sort = True)

In [31]:
#Create inverse emission probability dataframe
inv_emission_prob_table = pd.DataFrame(columns = vendor_list)
#Insert vendor name column
inv_emission_prob_table.insert(0, "drug", drug_list)
# Apply function to dataframe
inv_emission_prob_table.iloc[:,1:] = inv_emission_prob_table['drug'].apply(getVendorsForDrug)
# Sort by Vendor name and fill NA values with 0
inv_emission_prob_table = inv_emission_prob_table.fillna(0)

## Alternate method, using regular emission probabilities
Compute using $P(v~|~d) = \frac{P(d~|~v){P(v)}}{P(d)}$.

In [32]:
#Create emission probability dataframe
emission_prob_table = pd.DataFrame(columns = drug_list)
#Insert vendor name column
emission_prob_table.insert(0, "Vendor Name", vendor_list)
# Apply function to dataframe
emission_prob_table.iloc[:,1:] = emission_prob_table['Vendor Name'].apply(getDrugsForVendor)
# Sort by Vendor name and fill NA values with 0
emission_prob_table = emission_prob_table.fillna(0)

In [33]:
# Get probability for each vendor
vendor_probs = train_set['vendor_name'].value_counts()/np.sum(train_set['vendor_name'].value_counts())
vendor_probs = vendor_probs.sort_index()

# Get probability for each drug
drug_probs = train_set['drug_prediction'].value_counts()/np.sum(train_set['drug_prediction'].value_counts())
drug_probs = drug_probs.sort_index()

In [34]:
# Compute conditional probability table, using emission probabilities
vendor_probs_vec = np.array(vendor_probs).reshape(-1,1)
drug_probs_vec = np.array(drug_probs).reshape(1,-1)
factor_mat = np.matmul(vendor_probs_vec, 1/drug_probs_vec)
emission_prob_table.iloc[:,1:] = emission_prob_table.iloc[:,1:] * factor_mat

## Use Bayes' rule to predict vendor for each drug in sequence

In [74]:
# Conditional probability table
pvd = inv_emission_prob_table 
pvd = pvd.set_index('drug')

# Create dictionary with best prediction for each drug
pvd_dict = {drug : pvd.loc[drug].sort_values(ascending = False)[:10] for drug in pvd.index}

In [144]:
# Function to predict class for each drug in test set
def bayesPredict(obs_seq):
    preds = [pvd_dict[obs].index[0] for obs in obs_seq]
    top5_preds = [pvd_dict[obs].index[:5] for obs in obs_seq]
    top10_preds = [pvd_dict[obs].index[:10] for obs in obs_seq]
    return(preds, top5_preds, top10_preds)

preds, top5_preds, top10_preds = bayesPredict(d_test)

In [163]:
def bayesEval(obs_seq, hid_seq):   
    # Check if each prediction is in top5 and/or top10
    pairs = zip(obs_seq, hid_seq)
    inTop1 = [pair[1] == pvd_dict[pair[0]].index[0] for pair in pairs]
    pairs = zip(obs_seq, hid_seq)
    inTop5 = [pair[1] in pvd_dict[pair[0]].index[:5] for pair in pairs]
    pairs = zip(obs_seq, hid_seq)
    inTop10 = [pair[1] in pvd_dict[pair[0]].index[:10] for pair in pairs]
    return(inTop1, inTop5, inTop10)

inTop1, inTop5, inTop10 = bayesEval(d_test, v_test)

In [164]:
# Compile results in a dataframe
results = pd.DataFrame({'drug' : d_test, 'actual_vendor' : v_test, 'predicted_vendor' : preds,\
                        'inTop1':inTop1, 'inTop5':inTop5, 'inTop10': inTop10})

In [169]:
top1_acc = np.sum(results['inTop1'])/results.shape[0]
top5_acc = np.sum(results['inTop5'])/results.shape[0]
top10_acc = np.sum(results['inTop10'])/results.shape[0]
print('Top 1 accuracy :',round(top1_acc,3))
print('Top 5 accuracy :',round(top5_acc,3))
print('Top 10 accuracy:',round(top10_acc,3))

Top 1 accuracy : 0.103
Top 5 accuracy : 0.338
Top 10 accuracy: 0.56


In [170]:
results.head()

,drug,actual_vendor,predicted_vendor,inTop1,inTop5,inTop10
0,cocaine,DerSandmann,pastrydam,False,False,False
1,hashish,pillonpourvousservir,GreenFingers,False,False,True
2,hashish,pillonpourvousservir,GreenFingers,False,False,True
3,others,PHARMALABS,cashcow72,False,True,True
4,others,Gladyman,cashcow72,False,True,True


In [173]:
pvd_dict['cocaine'][:10]

pastrydam           0.082090
PopeyeUK            0.060276
Cocaineandbrandy    0.051091
drugkingz           0.046498
DrRubiks            0.040184
Boliko              0.039610
Mastery             0.039323
hiddenshadows       0.038462
therealLBZ          0.035017
DOPE_CHEF           0.033295
Name: cocaine, dtype: float64